<a href="https://colab.research.google.com/github/Vipulabhiram/GenAI_Assignment_1/blob/main/Assignment5(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers faiss-cpu

Ingestion


In [ ]:
sample_text="""Albert einstein was a theoretical physicist who developed the theory of relativity,
one of the two pillars of modern physiccs (alogside quantum mechanism). His work is also
know for its influence on the philosophy of science. He is best know to the general public
for his mass-energy equivalence formula E=mc^2."""

#This is a stage where we provide data on which we will be asking the querys on.

Embedding Function

In [ ]:
#Here we are importing AutoTokenizer it will find a tokenizer to convert it into tokens and Automodel it will finds a model for embedding.
#torch, numpy are Library used to work wth tensors(torch) and Vectors(numpy);
#no_grad function is used to inform the model not to train on the data that we provide and just to give responce for our query.
#mean(dim=1).squeeze is used to conver the last_hidden_state tensor into a single dimention vector.
#.numpy is used to convert it into numpy.
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
  tokens = tokenizer(text,return_tensors='pt',truncation=True,padding=True)

  with torch.no_grad():
    output = model(**tokens)
  return output.last_hidden_state.mean(dim=1).squeeze().numpy()

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
#We importing faiss to perform tasks of stroing, indexing and searching
#Embedding store the values of chunks after they are embedding by using for loop
#faiss.IndexFlatL2 is used to provide index for embedding.
import faiss

chunks = [sample_text]
embedding = [get_embedding(chunk) for chunk in chunks]

dim = len(embedding[0])
index = faiss.IndexFlatL2(dim)
index.add(np.array(embedding))

In [ ]:
#pipeline function is imported from transformers library used to assign NLP task to a particular modal
#indices referes the most chucnk after searchig on index by user query and stories that vector in retrived_text
#we will join evey item of it into a paragraph and make a content for the query given by user.
#now we will a prompt by giving user question and the content that we stored related  to that query and pass this prompt to modal to generate the answer and return it
from transformers import pipeline
qa_pipeline = pipeline("text2text-generation",model="google/flan-t5-small")

def retrive_and_answer(query,top_k=1):
  query_embedding = get_embedding(query).reshape(1,-1)
  _,indices = index.search(query_embedding,top_k)
  retrieved_text = [chunks[i] for i in indices[0]]
  context="".join(retrieved_text)

  prompt = f"context:{context} \n\n Question:{query} \nAnswer:"
  result = qa_pipeline(prompt,max_length=100)
  return result[0]['generated_text']

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
#Here we will give the user question and call retrive and answer function to generate the answer for our query and print both question and answer.
Question = "What is einstein famous for?"
Answer = retrive_and_answer(Question)
print("Q: ",Question)
print("A: ",Answer)

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q:  What is einstein famous for?
A:  his mass-energy equivalence formula E=mc2
